In [150]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

data_sets = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    
    for filename in filenames:
        print(filename)
        if filename == "test_essays.csv":
            test = pd.read_csv(os.path.join(dirname, filename))
        if filename == "train_essays.csv":
            train = pd.read_csv(os.path.join(dirname, filename))
        if filename == "train_essays_RDizzl3_seven_v2.csv":
            generated_text = pd.read_csv(os.path.join(dirname, filename))
        data_sets.append(pd.read_csv(os.path.join(dirname, filename)))
    print(f"{len(data_sets)} data sets loaded in.")
#sample_sub, training_prompts, test, train, _, _, _, generated_text = data_sets

0 data sets loaded in.
sample_submission.csv
train_prompts.csv
test_essays.csv
train_essays.csv
4 data sets loaded in.
train_essays_RDizzl3_seven_v2.csv
train_essays_7_prompts_v2.csv
train_essays_7_prompts.csv
train_essays_RDizzl3_seven_v1.csv
8 data sets loaded in.


In [151]:
print("Supplementary dataset features:")
print(generated_text.dtypes)
print("\nOriginal dataset features:")
print(train.dtypes)


Supplementary dataset features:
text     object
label     int64
dtype: object

Original dataset features:
id           object
prompt_id     int64
text         object
generated     int64
dtype: object


As we can see, we need to make it so that these share a universal format so that we can combine the data together.

In [130]:
# Rename the label column to match original dataset
generated_text.rename(columns={'label': 'generated'}, inplace=True)
generated_text

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
15866,"While some find the ""Face on Mars"" imaged by t...",1
15867,Limiting car usage has many benefits for moder...,1
15868,The Rise of Driverless Cars\n\nThe development...,1
15869,The Open Sea Beckons\n\nThe Seagoing Cowboys p...,1


In [131]:
# Drop the two columns we don't need for training
train.drop('id', axis='columns', inplace=True)
train.drop('prompt_id', axis='columns', inplace=True)
train

,text,generated
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0
...,...,...
1373,There has been a fuss about the Elector Colleg...,0
1374,Limiting car usage has many advantages. Such a...,0
1375,There's a new trend that has been developing f...,0
1376,As we all know cars are a big part of our soci...,0


In [132]:
training_set = pd.concat([generated_text, train])

In [133]:
training_set.describe()

,generated
count,17249.000000
mean,0.094150
std,0.292046
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [134]:
from sklearn.model_selection import train_test_split
import keras_nlp
import keras_core as keras

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
   

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping

In [135]:
isRealTest = True

if isRealTest:
    X_train = training_set['text']
    y_train = training_set['generated']
    
    X_test = test['text']
    X_testIDs = test['id']
    
    
else:
    X = training_set['text']
    y = training_set['generated']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [136]:
max_words = 10000
padding_length = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Tokenize them into vectors
# Each unique word represented by an integer
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad our data to ensure consistent length
X_train_padded = pad_sequences(X_train_sequences, maxlen=padding_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=padding_length, padding='post', truncating='post')

In [137]:
print(X_test)

0    Aaa bbb ccc.
1    Bbb ccc ddd.
2    CCC ddd eee.
Name: text, dtype: object


### Visualizing our preprocessing

Now we can see the three steps our data goes through


In [138]:
X_train[4]

4    Cars are a wonderful thing. They are perhaps o...
4    Cars are a wonderful thing. They are perhaps o...
Name: text, dtype: object

In [139]:
essay_number = 5

print("Step 1: Raw Text\n"+ X_train[essay_number])
print("\n\nStep 2: Tokens\n" + str(X_train_sequences[essay_number]))
print("\n\nStep 3: Padded Set\n" + str(X_train_padded[essay_number]))

print("Generated: " + str(y_train[essay_number]))

5    Step 1: Raw Text\nThe electrol college system ...
5    Step 1: Raw Text\nThe electrol college system ...
Name: text, dtype: object


Step 2: Tokens
[2, 5183, 34, 69, 8, 53, 243, 69, 20, 148, 17, 2, 228, 3, 1945, 44, 214, 55, 18, 234, 17, 2, 228, 3, 1945, 5, 55, 38, 52, 20, 37, 18, 13, 1939, 139, 11, 2, 90, 11, 5, 1, 9, 162, 24, 12, 5, 450, 29, 5, 1620, 7, 152, 233, 10, 43, 9, 90, 24, 402, 35, 2544, 162, 95, 9247, 2, 2, 410, 139, 203, 86, 62, 25, 37, 25, 37, 3, 1945, 5, 1301, 68, 39, 5553, 14, 211, 6, 82, 312, 5, 225, 4, 426, 2, 94, 275, 163, 86, 2, 5183, 1, 8, 243, 38, 4, 2, 5144, 10, 103, 42, 1254, 25, 28, 449, 20, 674, 402, 44, 214, 525, 28, 10, 177, 7, 152, 233, 9, 52, 128, 37, 18, 13, 139, 11, 2, 172, 90, 193, 121, 539, 57, 37, 6, 100, 2, 90, 24, 402, 68, 18, 118, 11, 55, 7, 152, 233, 10, 43, 9, 2, 27, 34, 1130, 4, 880, 90, 6, 2, 94, 229, 4, 27, 85, 8, 1263, 7, 607, 3, 780, 53, 55, 48, 7, 152, 233, 10, 43, 9, 2, 300, 4, 90, 25, 17, 2994, 3, 2, 229, 4, 1744, 4, 519, 25, 17, 26,

In [140]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=200),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss =tf.keras.losses.BinaryCrossentropy(), metrics=[BinaryAccuracy()])

In [141]:
y_train = y_train.astype(np.float32)
y_test  = y_test .astype(np.float32)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(X_train_padded, y_train, epochs=5, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/5
486/486 [==============================] - 55s 107ms/step - loss: 0.2625 - binary_accuracy: 0.9196 - val_loss: 0.2156 - val_binary_accuracy: 0.8046
Epoch 2/5
486/486 [==============================] - 51s 105ms/step - loss: 0.0790 - binary_accuracy: 0.9704 - val_loss: 0.1525 - val_binary_accuracy: 0.9507
Epoch 3/5
486/486 [==============================] - 52s 106ms/step - loss: 0.0532 - binary_accuracy: 0.9947 - val_loss: 0.0929 - val_binary_accuracy: 0.9919
Epoch 4/5
486/486 [==============================] - 52s 106ms/step - loss: 0.0394 - binary_accuracy: 0.9943 - val_loss: 0.0370 - val_binary_accuracy: 0.9959
Epoch 5/5
486/486 [==============================] - 55s 113ms/step - loss: 0.0211 - binary_accuracy: 0.9970 - val_loss: 0.0246 - val_binary_accuracy: 0.9959


In [142]:
if not isRealTest:
    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test_padded, y_test)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

In [143]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions on the test set
y_pred = model.predict(X_test_padded)

# Convert probabilities to class labels
y_pred_labels = np.round(y_pred)

1/1 [==============================] - 1s 532ms/step


In [144]:
if not isRealTest:
    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_labels)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Classification report
    class_report = classification_report(y_test, y_pred_labels)
    print("\nClassification Report:")
    print(class_report)

if isRealTest:
    submission = pd.DataFrame({'id': X_testIDs[:][1], 'generated': y_pred.flatten()})

In [145]:
submission.to_csv('/kaggle/working/delivery.csv', index=False)